# Agent with tools
This notebook illustrates how to build a simple agent capable of using tools. 
We'll use crewAI to illustrate how agentics implements logical transduction

## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [ ]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
  
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.11 environment at: /Users/gliozzo/miniforge3/envs/agentics
Audited 1 package in 62ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [ ]:
from crewai.tools import tool
from ddgs import DDGS

## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    

2025-09-09 11:33:36.214 | DEBUG    | agentics.core.llm_connections:<module>:90 - AGENTICS is connecting to the following LLM API providers:
2025-09-09 11:33:36.215 | DEBUG    | agentics.core.llm_connections:<module>:93 - 0 - WatsonX
2025-09-09 11:33:36.215 | DEBUG    | agentics.core.llm_connections:<module>:98 - 1 - Gemini
2025-09-09 11:33:36.215 | DEBUG    | agentics.core.llm_connections:<module>:102 - 2 - OpenAI
2025-09-09 11:33:36.215 | DEBUG    | agentics.core.llm_connections:<module>:104 - Please add API keys in .env file to add or disconnect providers.
2025-09-09 11:33:36.224 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-09 11:33:36.224 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'


## Define an Agent

Use crew AI agent directly create a document searcher agent, who is instructed to use the web_search tool defined above. Note that reasoning steps and other parameters can be customized. 


In [ ]:
from crewai import Agent
doc_agent = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=True,
    reasoning_steps=2,
    verbose=True,
    llm=get_llm_provider()
)

## Define a Task

In [ ]:
from crewai import Task

doc_task = Task(
    description="""Your task is to perform an extensive web search about
    the following question {question} and return a document providing answers to 
    the questions that explore several interesting aspects, each of them supported 
    by pertinent information from web search.  """,
    expected_output="""A markdown document with introduction and different sections""",
    agent=doc_agent,
    markdown=True
)

## Define a Crew

Crews are enviroments in which tasks are assigned to different agents following different policies. In this simple example we define a crew with a single agent and single task. 

In [ ]:
from crewai import Crew

crew = Crew(
    agents=[doc_agent],
    tasks=[doc_task],
    verbose=True,
)

## Kickoff the crew

Crews are ultimatelly analoguous to very sophisticated function definitions, whose inputs are json dictonaries. In order to put them in action you should kickoff them providing input parameters.

In the Agent Trace stack you can see the plan made by the agent and the different execution steps which might involve tool calls.

In [ ]:
result = crew.kickoff(inputs={"question": input("Ask your question> ") })

## Display the results as markdown

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result.raw))

## Define a conversational agent

In [ ]:
from crewai import Agent, Task, Crew
search_assistant = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=False,
    verbose=True,
    llm=get_llm_provider()
)

task = Task(
    description="""Your task is to help the user performing his / her web search on a deep subject.  
     You will start by asking the user what is his / her topic of interest. Then you will ask follow up question
     to clarity the user needs while executing your web search and provideing intermediate results to the user.
     When you think you have enough information you will provide your final output""",
    expected_output="""A markdown document that summarize first the user requests, then provide a detailed answers and finally report supporting evidence from web search""",
    agent=search_assistant,
    human_input=True,
    markdown=True
)

crew = Crew(
    agents=[search_assistant],
    tasks=[task],
    verbose=False,
)
result = crew.kickoff(inputs={})

2025-09-09 11:34:49.072 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-09 11:34:49.073 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Task: Your task is to help the user performing his / her web search on a deep subject.                         │
│       You will start by asking the user what is his / her topic of interest. Then you will ask follow up        │
│  question                                                                                                       │
│       to clarity the user needs while executing your web search and provideing intermediate results to the      │
│  user.                                                                                                          │
│       When you think you have enough information you will provide your final output                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Thought: Thought: I need to ask the user about their topic of interest to start the web search process.        │
│                                                                                                                 │
│  Using Tool: fetch_data_async                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"What is your topic of interest?\"}"                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'title': 'STLCC Topic Ideas', 'href':                                                                        │
│  'https://stlcc.edu/student-support/academic-success-and-tutoring/writing-center/writing-resources/topic-ideas  │
│  .aspx', 'body': 'What is a problem or solution you’ve been thinking about? What are you excited about? What    │
│  do you like to talk about and do? Below are some examples of broad topics and some ways of writing about the   │
│  topics based on some specific assignment types. Narrative: Write a story about a work-related event that       │
│  changed a way you think or behave; Observation: Using concrete, sensory language, describe an interesting      │
│  ...'}, {'title': 'Opencuny Picking a topic: What interests you? – Writing For Research at CSI', 'href':        │
│  'https://opencuny.org/writingforresearchcsi/picking-a-topic-what-interests-you/', 'body': 'Purpose: The first  │
│  step in writing a research paper is finding a good topic. Your topic will have to be something that falls      │
│  within the guidelines provided by your instructor, and should also be something that you find interesting and  │
│  would like to learn more about .'}, {'title': 'Develop a Research Topic or Question - Writing Resources',      │
│  'href': 'https://guides.library.jhu.edu/writing/research-process/develop', 'body': 'Three Approaches for       │
│  Developing a Topic. Approach #1: List Key Words of Interest. Make lists of concepts and topics you find        │
│  interesting .'}, {'title': 'Research Process: Select a Topic - LibGuides', 'href':                             │
│  'https://libguides.umflint.edu/research/topic', 'body': 'Write down any key words or concepts that may be of   │
│  interest to you . These terms can be helpful in your searching and used to form a more focused research topic  │
│  ...'}, {'title': 'How do I choose a good topic for my research paper? - FAQ', 'href':                          │
│  'https://tnstate.libanswers.com/faq/292959', 'body': 'May 21, 2024 — You want a focused topic that will make   │
│  a good paper. Here are some things to consider: Make sure your topic meets the assignment requirements .'},    │
│  {'title': '150 Research Paper Topics', 'href': 'https://www.mometrix.c...                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"agentic platforms\"}"                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'title': 'Olive Technologies Top Agentic AI Platforms in 2025: The Ultimate Guide for Businesses | Olive     │
│  Technologies', 'href':                                                                                         │
│  'https://olive.app/blog/top-agentic-ai-platforms-in-2025-the-ultimate-guide-for-businesses/', 'body': 'April   │
│  1, 2025 - Discover the top agentic ai platforms in 2025! Compare the best systems for enterprise, startups,    │
│  and smb. Find the right agentic platform .'}, {'title': 'Moveworks 11 Leading Agentic AI Tools for Businesses  │
│  | Moveworks', 'href': 'https://www.moveworks.com/us/en/resources/blog/agentic-ai-tools-for-business', 'body':  │
│  'March 7, 2025 - Because agentic tools are so flexible, instead of creating separate tools for every           │
│  challenge, agentic AI platforms can handle various business needs across departments—like IT service           │
│  management, HR support, finance operations, and sales enablement.'}, {'title': 'Medium Top 5 Agentic AI        │
│  Frameworks to Watch in 2025 | by Lekha Priya | Medium', 'href':                                                │
│  'https://lekha-bhan88.medium.com/top-5-agentic-ai-frameworks-to-watch-in-2025-9d51b2b652c0', 'body': 'January  │
│  9, 2025 - The evolution of Agentic AI is unlocking new frontiers in autonomous systems and intelligent         │
│  agents. As we enter 2025, developers and enterprises are looking for frameworks that not only streamline AI    │
│  workflows but also enable complex decision-making and collaboration .'}, {'title': 'MSSP Alert Dataminr Adds   │
│  Agentic AI to Cybersecurity Platforms | MSSP Alert', 'href':                                                   │
│  'https://www.msspalert.com/brief/dataminr-adds-agentic-ai-to-cybersecurity-platforms', 'body': '1 day ago -    │
│  Dataminr has introduced Agentic AI to its Pulse for Cyber Risk API, bringing real-time intelligence tools      │
│  like Live Briefs, Intel Agents, and Cyber Anomaly Alerts directly into widely used cybersecurity platforms.    │
│  The goal is to give security teams faster, more contextual insight into unfolding ...'}, {'title': 'C3 AI C3   │
│  Agentic AI Platform: Enterprise and IoT Applications | C3 AI', 'href':                                         │
│  'https://c3.ai/c3-agentic-ai-platform/', 'body': 'March 17, 2025 - ...                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Thought: Thought: I have found some information about agentic platforms, but I need to organize it and         │
│  provide a clear summary to the user.                                                                           │
│                                                                                                                 │
│  Using Tool: fetch_data_async                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"agentic platforms definition and examples\"}"                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'title': 'What is an Agentic Platform and Its Essential Capabilities?', 'href':                              │
│  'https://www.krista.ai/what-is-an-agentic-platform-and-its-essential-capabilities/', 'body': 'Nov 20, 2024 —   │
│  An agentic platform is designed to create autonomous AI agents —intelligent, software-driven tools that        │
│  execute tasks traditionally handled by ...'}, {'title': 'What Is Agentic AI? Definition, Types, Examples',     │
│  'href': 'https://www.workday.com/en-us/topics/ai/agentic-ai.html', 'body': 'Agentic systems are designed to    │
│  work together . One agent may detect an issue, while others handle communications, inventory updates, or       │
│  policy checks, each ...'}, {'title': 'What Is Agentic AI? | IBM', 'href':                                      │
│  'https://www.ibm.com/think/topics/agentic-ai', 'body': 'Agentic AI is an artificial intelligence system that   │
│  can accomplish a specific goal with limited supervision. It consists of ai agents—machine learning ...'},      │
│  {'title': 'Agentic Frameworks: The Systems Used to Build AI Agents', 'href':                                   │
│  'https://www.moveworks.com/us/en/resources/blog/what-is-agentic-framework', 'body': 'Feb 14, 2025 — An         │
│  agentic framework supports the creation of AI agents and informs how they interact with each other to provide  │
│  automation.'}, {'title': 'Agentic: Definition, meaning, and real-world examples', 'href':                      │
│  'https://www.eesel.ai/blog/agentic-definition', 'body': 'Agentic AI acts and thinks independently to help      │
│  businesses automate complex tasks, improve service, and scale faster without constant human guidance.'},       │
│  {'title': 'What is Agentic AI? Definition, Best Practices, Examples', 'href':                                  │
│  'https://www.atscale.com/glossary/agentic-ai/', 'body': 'Agentic AI refers to artificial intelligence systems  │
│  that possess agency, the ability to reason, make decisions, and take action autonomously within a defined      │
│  ...'}, {'title': 'What Is Agentic AI?', 'href': 'https://blogs.nvidia.com/blog/what-is-agentic-ai/', 'body':   │
│  'Oct 22, 2024 — Agentic AI uses sophisticated reasoning and iterative planning to autonomously solv...         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯